In [ ]:
%load_ext autoreload
%autoreload 2
%cd /mnt/c/Users/resha/Documents/Github/balancing_framework/

import pickle
import pandas as pd
import numpy as np
import json
import time
import argparse
from tqdm import tqdm

from framework import run_measurements, viz
from fracdiff import frac_diff_bestd
from monash_data_utils import convert_tsf_to_dataframe, monash_df_to_gluonts_train_datasets
import os

from statsmodels.tsa.stattools import adfuller

def run_adf(series):
    adf_chunk_size = 100_000
    num_stat = (0,0) # number of stationary windows, total number of windows
    p_values = []
    for i in range(0, len(series), adf_chunk_size):
        data_chunk = series.dropna()[i:i+adf_chunk_size]
        if data_chunk.nunique()==1:
            print(f'series has only one unique value: {series.iloc[0]}')
            continue
        adf_result = adfuller(data_chunk) 
        # print(f'{i} p-value={adf_result[1]}, lags={adf_result[2]}')
        num_stat = (num_stat[0], num_stat[1]+1)
        p_values.append(adf_result[1])
        if adf_result[1] < 0.05:
            num_stat = (num_stat[0]+1, num_stat[1])
    # if more than 50% of the p-values are above 0.05, then the data is not stationary
    stationary = num_stat[0] >= num_stat[1]/2
    return stationary


/mnt/c/Users/resha/Documents/Github/balancing_framework


In [67]:
df.dtypes['holiday']

dtype('O')

In [68]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
dataset = fetch_ucirepo(id=492) 
  
# data (as pandas dataframes) 
X = dataset.data.features 
y = dataset.data.targets 

# Concatenate features and targets into a single DataFrame
df = pd.concat([X, y], axis=1)

# Step 1: Replace '?' with NaN across the entire DataFrame
df.replace('?', np.nan, inplace=True)

# Step 2: Convert data columns to a numeric type, excluding Date and Time
for col in df.columns:
    if col in ['date', 'Time', 'DateTime', 'date_time'] or df[col].dtype == 'O': continue
    df[col] = pd.to_numeric(df[col])

# Step 3: Combine Date and Time into a single datetime index
# df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time']) # dayfirst=True
# df.set_index('datetime', inplace=True)
# df.drop(['Date', 'Time'], axis=1, inplace=True)

# Step 4: Impute the missing values
# We will use linear interpolation, which is a good choice for continuous time series data.
# This method fills a gap by drawing a straight line between the known values before and after the NaN.

# print("\nNumber of NaNs before interpolation:")
# print(df.isnull().sum())
nansbef = df.isnull().sum().sum()
df.interpolate(method='linear', inplace=True)
# You can also use other methods:
# df.fillna(method='ffill', inplace=True) # Forward fill
# Verify the result
# print("DataFrame info after interpolation:")
# print("\nNumber of NaNs after interpolation:")
# print(df.isnull().sum())
print(f'{df.isnull().sum().sum()} NaNs were imputed out of {nansbef} total NaNs')

print(df.info())

48143 NaNs were imputed out of 48143 total NaNs
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   holiday              61 non-null     object 
 1   temp                 48204 non-null  float64
 2   rain_1h              48204 non-null  float64
 3   snow_1h              48204 non-null  float64
 4   clouds_all           48204 non-null  int64  
 5   weather_main         48204 non-null  object 
 6   weather_description  48204 non-null  object 
 7   date_time            48204 non-null  object 
 8   traffic_volume       48204 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 3.3+ MB
None


In [69]:
for col in df.columns:
    if df[col].dtype == 'O': continue
    print(col, run_adf(df[col]))

temp True
rain_1h True
snow_1h True
clouds_all True
traffic_volume True
